In [1]:
import sys
sys.path.append('..')

import re
import os
import re
import json
import glob
import pandas as pd
from pathlib import Path
from strsimpy.levenshtein import Levenshtein
from src.metrics.ocr_eval import *

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
DOC_TYPE = 'sim'
DATA_DIR = Path(f"../data/processed/{DOC_TYPE}")
TRAIN_DIR = "train"
TEST_DIR = "test"
ANNOT_DIR = Path("../data/interim/2022-01-06_combined_orc_kyc/")
# ANNOT_DIR = Path("../data/tmp/")
ANNOT_CSV_FILE = f"{DOC_TYPE}_annotated.csv"
RES_DIR = Path("../data/res")
RES_CSV_FILE = f"{DOC_TYPE}_inference.csv"
GCS_LINK = f"gs://som-datasets/interim/2022-01-06_combined_orc_kyc/SIM-OLD/"
# GCS_LINK = f"gs://som-datasets/processed/{DOC_TYPE}/train/images/"

In [4]:
annot_df = pd.read_csv(str(ANNOT_DIR/ANNOT_CSV_FILE))
annot_df

,Unnamed: 0,Unnamed: 0.1,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,doc,...,tgl_lahir,jenis_kelamin,tempat,alamat,kel_desa,kecamatan,agama,pekerjaan,kewarganegaraan,berlaku_hingga
0,0,54,202106100907_15a.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,946,0,NaN,SIM,...,21-12-1975,NaN,NaN,PERMATA BALOI C2/12A LUBUK BAJA KOTA BATAN,NaN,NaN,NaN,WIRASWASTA,KEPRI,NaN
1,1,55,202106100912_5.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,948,0,NaN,SIM,...,14-09-1969,NaN,SURABAYA,"PETEMON 2/60-A,RT04 RW09 PETEMON, SAWAHAN KOTA...",NaN,NaN,NaN,SWASTA,JATIM,07-09-2024
2,2,56,202106100923_14a.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,949,0,NaN,SIM,...,11-12-1956,NaN,PACITAN,JOJORAN 3-D/48 RT 010 RW 012 SURABAYA,NaN,NaN,NaN,SWASTA,NaN,11-12-2021
3,3,57,All_KYC_Docs_117c.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,950,0,NaN,SIM,...,09-09-1962,NaN,NaN,RAJAWALI N004 RT 22 LIR IT II KOTA PALEMBANG,NaN,NaN,NaN,SWASTA,SUMSEL,18-12-2024
4,4,58,All_KYC_Docs_54.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,952,0,NaN,SIM,...,01-08-1983,NaN,NaN,BURCIT INDIAN RAYA SUKAJ AD1,NaN,NaN,NaN,WIRASWASTA,NaN,02-08-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,610,DL111.png,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,NaN,SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,251,611,DL112.png,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,NaN,SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,252,766,DL30.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,NaN,SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,253,767,DL39.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,NaN,SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
annot_df.fillna('', inplace=True)
annot_df = annot_df.astype(str)
annot_df

,Unnamed: 0,Unnamed: 0.1,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,doc,...,tgl_lahir,jenis_kelamin,tempat,alamat,kel_desa,kecamatan,agama,pekerjaan,kewarganegaraan,berlaku_hingga
0,0,54,202106100907_15a.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,946,0,,SIM,...,21-12-1975,,,PERMATA BALOI C2/12A LUBUK BAJA KOTA BATAN,,,,WIRASWASTA,KEPRI,
1,1,55,202106100912_5.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,948,0,,SIM,...,14-09-1969,,SURABAYA,"PETEMON 2/60-A,RT04 RW09 PETEMON, SAWAHAN KOTA...",,,,SWASTA,JATIM,07-09-2024
2,2,56,202106100923_14a.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,949,0,,SIM,...,11-12-1956,,PACITAN,JOJORAN 3-D/48 RT 010 RW 012 SURABAYA,,,,SWASTA,,11-12-2021
3,3,57,All_KYC_Docs_117c.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,950,0,,SIM,...,09-09-1962,,,RAJAWALI N004 RT 22 LIR IT II KOTA PALEMBANG,,,,SWASTA,SUMSEL,18-12-2024
4,4,58,All_KYC_Docs_54.jpg,False,SIM-NEW,gs://som-datasets/interim/2022-01-06_combined_...,952,0,,SIM,...,01-08-1983,,,BURCIT INDIAN RAYA SUKAJ AD1,,,,WIRASWASTA,,02-08-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,610,DL111.png,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,
251,251,611,DL112.png,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,
252,252,766,DL30.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,
253,253,767,DL39.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,


In [6]:
def load_df(DATA_DIR, FILE, fields):
    
    def swap(a, b):
        return b, a
    
    fields += ['tempat_tgl_lahir']
    max_field_lenght = max([len(i) for i in fields])
    df_dict = {
        'file_name': [],
        'image_file': [],
        'json_file': [],
        'gcs_link': [],
        'key_text': [],
        'key_x1': [],
        'key_y1': [],
        'key_x2': [],
        'key_y2': [],
        'key_x3': [],
        'key_y3': [],
        'key_x4': [],
        'key_y4': [],
        'key_confidence': [],
        'value_text': [],
        'value_x1': [],
        'value_y1': [],
        'value_x2': [],
        'value_y2': [],
        'value_x3': [],
        'value_y3': [],
        'value_x4': [],
        'value_y4': [],
        'value_confidence': [],
        'image_width': [],
        'image_height': []
    }
    files = glob.glob(str(DATA_DIR/FILE/"images/*.jpeg")) + glob.glob(str(DATA_DIR/FILE/"images/*.jpg")) + glob.glob(str(DATA_DIR/FILE/"images/*.webp")) + glob.glob(str(DATA_DIR/FILE/"images/*.png"))
    for file in files:
        file_name = file.split('/')[-1].split('.')[0]
        if not os.path.exists(str(DATA_DIR/FILE/f'json/{file_name}_document_ai.json')):
            continue
        json_file = str(DATA_DIR/FILE/f'json/{file_name}_document_ai.json')
        with open(json_file, 'r') as f:
            data = json.load(f)
        for ind, page in enumerate(data['form_field']):
            image_width, image_height = data['image_width'][ind], data['image_height'][ind]
            for key_value in page:
                is_present = False
                key, value = key_value['key'], key_value['value']
                key_text, value_text = re.sub(r'[\s/:.,]', ' ', key['text'].strip().lower()).strip(), re.sub(r'[\s/:.,]', ' ', value['text'].strip().lower()).strip()
                for field in fields:
                    if field in key_text:
                        is_present = True
                    if field in value_text:
                        key, value = swap(key, value)
                        key_text, value_text = swap(key_text, value_text)
                        is_present = True
                if not is_present or len(key_text) > max_field_lenght:
                    continue
                df_dict['file_name'].append(file_name)
                df_dict['image_file'].append(file)
                df_dict['json_file'].append(json_file)
                df_dict['gcs_link'].append(GCS_LINK + file.split('/')[-1])
                df_dict['key_text'].append(key_text)
                df_dict['key_x1'].append(key['bbox']['x1'])
                df_dict['key_y1'].append(key['bbox']['y1'])
                df_dict['key_x2'].append(key['bbox']['x2'])
                df_dict['key_y2'].append(key['bbox']['y2'])
                df_dict['key_x3'].append(key['bbox']['x3'])
                df_dict['key_y3'].append(key['bbox']['y3'])
                df_dict['key_x4'].append(key['bbox']['x4'])
                df_dict['key_y4'].append(key['bbox']['y4'])
                df_dict['key_confidence'].append(key['confidence'])
                df_dict['value_text'].append(value_text)
                df_dict['value_x1'].append(value['bbox']['x1'])
                df_dict['value_y1'].append(value['bbox']['y1'])
                df_dict['value_x2'].append(value['bbox']['x2'])
                df_dict['value_y2'].append(value['bbox']['y2'])
                df_dict['value_x3'].append(value['bbox']['x3'])
                df_dict['value_y3'].append(value['bbox']['y3'])
                df_dict['value_x4'].append(value['bbox']['x4'])
                df_dict['value_y4'].append(value['bbox']['y4'])
                df_dict['value_confidence'].append(value['confidence'])
                df_dict['image_width'].append(image_width)
                df_dict['image_height'].append(image_height)
    return pd.DataFrame.from_dict(df_dict)

In [7]:
df = load_df(DATA_DIR, TRAIN_DIR, [re.sub(r'[\s:,.]', ' ', i.strip().lower().replace('_', ' ')) for i in cols[DOC_TYPE]])
df['key_text'] = df['key_text'].apply(lambda x: x.replace(' ', '_'))
df[['key_text', 'value_text']]

,key_text,value_text
0,pekerjaan,pengemudi
1,tgl_lahir,03-02-1991
2,tempat_&,baturaja
3,pekerjaan,wiraswasta
4,nama,r zaky miftahul fasa
...,...,...
131,pekerjaan,wi aswasta
132,nama,yulinda putri
133,nama,sugianto
134,nama,taufan irawan


In [12]:
df[df['file_name'] == 'DL1'][['key_text', 'value_text']]

,key_text,value_text
3,pekerjaan,wiraswasta
4,nama,r zaky miftahul fasa
5,tgl_lahir,16-04-1991


In [13]:
special_tags = ['tempat_tgl_lahir']
res_df = pd.DataFrame()
for file_name in df['file_name'].unique().tolist():
    temp_df = df[df['file_name'] == file_name]
    res = {
        i: '' for i in cols[DOC_TYPE]
    }
    res['gcs_link'], res['file_name'] = '', ''
    for ind, row in temp_df.iterrows():
        if row['key_text'] in res.keys() or row['key_text'] in special_tags:
            if row['key_text'] == 'tempat_tgl_lahir':
                tempat_tgl_lahir = row['value_text'].split(' ')
                if len(tempat_tgl_lahir) == 1:
                    tempat, tgl_lahir = tempat_tgl_lahir[0], tempat_tgl_lahir[0]
                else:
                    tempat, tgl_lahir = ' '.join(row['value_text'].split(' ')[:-1]), row['value_text'].split(' ')[-1]
                res['tempat'] = tempat
                res['tgl_lahir'] = tgl_lahir
            else:
                res[row['key_text']] = row['value_text']
        res['gcs_link'] = row['gcs_link']
        res['file_name'] = row['file_name']
    res_df = pd.concat([res_df, pd.json_normalize(res)])
res_df = res_df.reset_index()
res_df.drop(['index'], inplace=True, axis=1)

In [14]:
res_df

,nama,tgl_lahir,jenis_kelamin,tempat,alamat,kel_desa,kecamatan,agama,pekerjaan,kewarganegaraan,berlaku_hingga,gcs_link,file_name
0,,03-02-1991,,,,,,,pengemudi,,,gs://som-datasets/interim/2022-01-06_combined_...,DL14
1,r zaky miftahul fasa,16-04-1991,,,,,,,wiraswasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL1
2,yanuar heru p,13-01-1984,,,,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL10
3,,04-04-1980,,,nama dano yusman jl cipinang muara rt05 06 ...,,,,karyawan,,,gs://som-datasets/interim/2022-01-06_combined_...,DL13
4,budi hadi cahyanto,12-11-1980,,,jl moh thamrin nort 04 rw 03,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL16
5,,20-11-1991,,,nama ahmad kanzu syauqi firdaus ds sb dadi...,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL17
6,,01-09-1974,,,,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL19
7,petrus mele,,,,nama,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL20
8,,25-02-1975,,,,,,,swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL22
9,pulih hosea,25-09-1987,,,,,,,peg swasta,,,gs://som-datasets/interim/2022-01-06_combined_...,DL23


In [15]:
res_df.to_csv(str(RES_DIR/RES_CSV_FILE), index=False)

In [16]:
res_df = pd.read_csv(str(RES_DIR/RES_CSV_FILE))
res_df

,nama,tgl_lahir,jenis_kelamin,tempat,alamat,kel_desa,kecamatan,agama,pekerjaan,kewarganegaraan,berlaku_hingga,gcs_link,file_name
0,NaN,03-02-1991,NaN,NaN,NaN,NaN,NaN,NaN,pengemudi,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL14
1,r zaky miftahul fasa,16-04-1991,NaN,NaN,NaN,NaN,NaN,NaN,wiraswasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL1
2,yanuar heru p,13-01-1984,NaN,NaN,NaN,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL10
3,NaN,04-04-1980,NaN,NaN,nama dano yusman jl cipinang muara rt05 06 ...,NaN,NaN,NaN,karyawan,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL13
4,budi hadi cahyanto,12-11-1980,NaN,NaN,jl moh thamrin nort 04 rw 03,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL16
5,NaN,20-11-1991,NaN,NaN,nama ahmad kanzu syauqi firdaus ds sb dadi...,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL17
6,NaN,01-09-1974,NaN,NaN,NaN,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL19
7,petrus mele,NaN,NaN,NaN,nama,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL20
8,NaN,25-02-1975,NaN,NaN,NaN,NaN,NaN,NaN,swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL22
9,pulih hosea,25-09-1987,NaN,NaN,NaN,NaN,NaN,NaN,peg swasta,NaN,NaN,gs://som-datasets/interim/2022-01-06_combined_...,DL23


In [17]:
res_df.fillna('', inplace=True)
# res_df['nik'] = res_df['nik'].astype(str)
res_df = res_df.astype(str)

In [18]:
a = res_df['gcs_link'].unique().tolist()
b = annot_df['gcs_link'].unique().tolist()

In [19]:
a = set(a)
b = set(b)

In [20]:
a-b

{'gs://som-datasets/interim/2022-01-06_combined_orc_kyc/SIM-OLD/DL31_2.jpeg'}

In [21]:
df = pd.merge(annot_df, res_df, on='gcs_link', suffixes=("_annotated", ""))
df

,Unnamed: 0,Unnamed: 0.1,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,doc,...,jenis_kelamin,tempat,alamat,kel_desa,kecamatan,agama,pekerjaan,kewarganegaraan,berlaku_hingga,file_name
0,18,72,202106100907_13.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,760,0,,SIM,...,,,,,,,swasta,,,202106100907_13
1,41,95,202106100918_2.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,789,0,,SIM,...,,,,,,,peg negeri,,,202106100918_2
2,48,102,202106100924_2.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,1,413 Request Entity Too Large,SIM,...,,,,,,,swasta,,,202106100924_2
3,51,105,202106100925_3a.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,811,0,,SIM,...,,,jl mangga vie 400,,,,swasta,,,202106100925_3a
4,78,132,All_KYC_Docs_50a.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,peg negeri,,,All_KYC_Docs_50a
5,83,137,All_KYC_Docs_72.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,swasta,,,All_KYC_Docs_72
6,86,140,All_KYC_Docs_80.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,wi aswasta,,,All_KYC_Docs_80
7,88,142,All_KYC_Docs_9.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,All_KYC_Docs_9
8,91,145,Copy_STNK_&_KTP_0b.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,,,,Copy_STNK_&_KTP_0b
9,95,149,All_KYC_Docs_108.jpg,False,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM,...,,,,,,,swasta,,,All_KYC_Docs_108


In [22]:
df[['pekerjaan', 'pekerjaan_annotated']]

,pekerjaan,pekerjaan_annotated
0,swasta,SWASTA
1,peg negeri,PEGNEGERI
2,swasta,
3,swasta,SWASTA
4,peg negeri,
5,swasta,
6,wi aswasta,
7,,
8,,
9,swasta,


In [51]:
df.columns.to_list()

['Unnamed: 0',
 'Unnamed: 0.1',
 'image',
 'is_colored',
 'file_type',
 'gcs_link',
 'analysis_id',
 'is_invalid',
 'annotation_error',
 'doc',
 'nama_annotated',
 'tgl_lahir_annotated',
 'jenis_kelamin_annotated',
 'tempat_annotated',
 'alamat_annotated',
 'kel_desa_annotated',
 'kecamatan_annotated',
 'agama_annotated',
 'pekerjaan_annotated',
 'kewarganegaraan_annotated',
 'berlaku_hingga_annotated',
 'nama',
 'tgl_lahir',
 'jenis_kelamin',
 'tempat',
 'alamat',
 'kel_desa',
 'kecamatan',
 'agama',
 'pekerjaan',
 'kewarganegaraan',
 'berlaku_hingga',
 'file_name']

In [52]:
df[['pekerjaan', 'pekerjaan_annotated']]

,pekerjaan,pekerjaan_annotated
0,swasta,SWASTA
1,peg negeri,PEGNEGERI
2,swasta,
3,swasta,SWASTA
4,peg negeri,
5,swasta,
6,wi aswasta,
7,,
8,,
9,swasta,


In [53]:
ldist = Levenshtein()

def apply_levenshtein(x):
    for field in cols[DOC_TYPE]:
        fieldstr = str(x[field]).lower().strip()
        annotstr = str(x[f"{field}_annotated"]).lower().strip()
        x[f"{field}_dist"] = int(ldist.distance(fieldstr, annotstr))

        # zero division error corrections. if both annotated and source are blank. we consider it as 0% error.
        if x[f"{field}_dist"] == 0:
            x[f"{field}_err"] = 0
        else:
            x[f"{field}_err"] = min(x[f"{field}_dist"] * 100 / len(annotstr) if len(annotstr) > 0 else 100, 100)
    return x

df = df.apply(apply_levenshtein, axis=1)

In [54]:
metrics = df[[f + "_err" for f in cols[DOC_TYPE]]].mean()
for k,v in metrics.iteritems():
    print(f"{k}\t{v:.2f}")

nama_err	52.50
tgl_lahir_err	75.00
jenis_kelamin_err	0.00
tempat_err	7.50
alamat_err	36.61
kel_desa_err	0.00
kecamatan_err	0.00
agama_err	0.00
pekerjaan_err	82.78
kewarganegaraan_err	0.00
berlaku_hingga_err	7.50
